# Visualize Calibration Process

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('rb') as fh: # has loss values for shortest path
    full_set = pickle.load(fh)
full_ods = stochastic_optimization.match_results_to_ods_w_year(full_set)
# for adding coordinates to the viz part
with (config['cycleatl_fp']/"rdp.pkl").open('rb') as fh:
    coords = pickle.load(fh)

with (config['cycleatl_fp']/"trips_2.pkl").open('rb') as fh:
    trips = pickle.load(fh)
with (config['cycleatl_fp']/"users_2.pkl").open('rb') as fh:
    users = pickle.load(fh)
trips.set_index('tripid',inplace=True)
users.set_index('userid',inplace=True)
trips = trips.loc[list(full_set.keys())]
users = users.loc[users.index.isin(set(list(trips['userid'])))]
user_map = trips['userid'].to_dict()

In [ ]:
#bring in a calibration result


In [ ]:
x = [(userid,len(item['past_vals']),item['results'].nfev,item['settings']['options']['popsize'],item['results'].success) for userid, item in calibration_results.items()]
# y = [ for _, item in calibration_results.items()]
x = [y for y in x if y[2] == y[1]]
x

In [ ]:
# these are the betas we need
list_of_betas = calibration_results['813_jaccard_buffer_mean']['past_vals']
betas_tup = calibration_results['813_jaccard_buffer_mean']['betas_tup']
set_to_zero = calibration_results['813_jaccard_buffer_mean']['set_to_zero']
set_to_inf = calibration_results['813_jaccard_buffer_mean']['set_to_inf']

# trips[trips['userid']==813]
tripid = 8344

o = full_set[tripid]['origin_node']
d = full_set[tripid]['destination_node']
year = full_set[tripid]['trip_start_time']

# if infra is on street (i.e., the link is still traversable but the impedance doesn't apply)
links.loc[links['year'] > year,set_to_zero] = 0 
# if it's off-street then assign it a very high cost
links.loc[(links['year'] > year) & (links.loc[:,set_to_inf]==1).any(axis=1),'link_cost_override'] = True

all_shortest_paths = []
for betas in tqdm(list_of_betas):

    turn_G_copy = turn_G.copy()        

    # run network update
    rustworkx_routing_funcs.impedance_update(betas,betas_tup,
            stochastic_optimization.link_impedance_function,
            'travel_time_min',
            stochastic_optimization.turn_impedance_function,
            links,turns,turn_G_copy)

    # re-add virtual links
    rustworkx_routing_funcs.add_virtual_edges([o],[d],links,turns,turn_G_copy)

    path_lengths, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths((o,d),turn_G_copy)
    all_shortest_paths.append(shortest_paths[0])

In [ ]:
all_shortest_paths = [MultiLineString([geo_dict[x[0]] for x in shortest_path]) for shortest_path in all_shortest_paths]

In [ ]:
pop_size = 5
new = []
for x in range(pop_size):
    new.append(all_shortest_paths[x::pop_size])

In [ ]:
new[0][0]

In [ ]:
# for idx, iteration in enumerate(new):
#     for mls, iteration:
#         for line in mls:
#             x, y = line.xy
#             ax.plot(x,y,colors)

In [ ]:
# for calibration_iter in all_shortest_paths:

    

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

# Function to plot a GeoSeries and save the plot
def plot_geoseries(geoseries,other_geoseries,i,past_val):
    fig, ax = plt.subplots(figsize=(20, 20))
    #cx.add_basemap(ax)
    other_geoseries.plot(ax=ax,color='blue',weight=1)
    geoseries.plot(ax=ax,color='red')
    ax.set_title(f"Iter:{i} Overlap Function:{past_val}")
    ax.set_axis_off()
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close()
    return img_bytes.getvalue()




In [ ]:
# need to generate routes using the past betas for only one trip
